In [1]:
import cv2
import uuid
import os
import time

In [2]:
import tensorflow

# Labels

In [3]:
labels = ['1', '2', '3', '4','5', '6', '7', '8','9','A', 'B', 'C', 'D','E', 'F', 'G', 'H','I', 'J', 'K', 'L','M', 'N', 'O', 'P','Q','R', 'S', 'T', 'U','V', 'W', 'X', 'Y', 'Z']
# labels = ['1', '2', '3', '4']

# Folder path

In [4]:
root_path=os.path.join('ISL','workspace')
video_path=os.path.join(root_path,'video')
img_path=os.path.join(root_path,'image','collected_img')
aug_img_path=os.path.join(root_path,'image','augmented_img')
train_path=os.path.join(root_path,'train')
test_path=os.path.join(root_path,'test')
validate_path=os.path.join(root_path,'validate')
model_path=train=os.path.join(root_path,'model')

In [5]:
if not os.path.exists(root_path):
    if os.name == 'posix':
        !mkdir -p {root_path}
    if os.name == 'nt':
        !mkdir {root_path}
        
if not os.path.exists(img_path):
    if os.name == 'posix':
        !mkdir -p {img_path}
    if os.name == 'nt':
        !mkdir {img_path}

In [7]:
for label in labels:
    l_path = os.path.join(video_path, label)
    if not os.path.exists(l_path):
        !mkdir {l_path}

# Video capture

In [9]:
from cv2 import VideoWriter_fourcc

# Define the duration in seconds of the video capture here
capture_duration = 10
name="M_front"

for label in labels:
    print("Video for label {}".format(label))
    time.sleep(15)
    print("start now....")
    webcam = cv2.VideoCapture(0)
    file_path=os.path.join(video_path,label,label+"-"+name+'.{}.avi'.format(str(uuid.uuid1())))
    video = cv2.VideoWriter(file_path,VideoWriter_fourcc(*'XVID'), 20.0, (640,480))

    start_time = time.time()
    while( int(time.time() - start_time) < capture_duration ):
        ret, frame = webcam.read()
        if ret==True:
            cv2.imshow('WEBCAM',frame)
            video.write(frame)
            #if cv2.waitKey(1) & 0xFF == ord('q'):
            #    break
        else:
            break

# Release everything if job is finished
webcam.release()
video.release()
cv2.destroyAllWindows()

Video for label T
start now....
Video for label W
start now....


In [15]:
for label in labels:
    l_path = os.path.join(img_path, label)
    if not os.path.exists(l_path):
        !mkdir {l_path}

In [16]:
for label in labels:
    l_path = os.path.join(aug_img_path, label)
    if not os.path.exists(l_path):
        !mkdir {l_path}

# Image capture

In [ ]:
for label in labels:
    # path of each video and image folder
    label_video_path=os.path.join(video_path,label)
    label_img_path=os.path.join(img_path,label)
    
    # getting name of each file in that directory 
    for filename in os.listdir(label_video_path):
        # image name
        img_root_name=str(filename).replace(".avi","")
        # path of the video file
        file_path = os.path.join(label_video_path, filename)
        webcam = cv2.VideoCapture(file_path)
        currentframe = 0
        
        while(True):
            # reading from frame
            ret,frame = webcam.read()
            if ret:
                # if video is still left continue creating images
                name = img_root_name + str(currentframe) + '.jpg'
                print ('Creating...' + name)
                
                # writing the extracted images
                cv2.imwrite(os.path.join(label_img_path,name), frame)
                # show how many frames are created
                currentframe += 1
            else:
                break
webcam.release()
cv2.destroyAllWindows()

In [ ]:
# for label in labels:
#     label_img_path=os.path.join(train_path)
#     lst=list(os.listdir(label_img_path))
#     for i in range(0,len(lst),3):
#         try: 
#             os.remove(os.path.join(label_img_path,lst[i]))
#         except: 
#             print("error")
#             pass

# Dataset augmentation

In [ ]:
# !pip install imgaug

In [17]:
import imgaug.augmenters as iaa

In [18]:
aug_lst=[iaa.Fliplr(1),iaa.GaussianBlur(sigma=(0, 1)),iaa.LinearContrast((0.75, 1.5)),
     iaa.Multiply((0.8, 1.2), per_channel=1), iaa.Flipud(1),iaa.Sharpen(alpha=(0, 1.0),lightness=(0.75, 1.5)),
     iaa.Invert(1, per_channel=True),iaa.Crop(px=(0, 10))]

In [19]:
for label in labels:
    label_img_path=os.path.join(img_path,label)
    
    # different augmentation
    for category in aug_lst:
        print("category : ",category)
        seq=iaa.Sequential([category])
        
        # iaa.Fliplr(1) -> .Fliplr
        category=str(category)
        sub_nm=category[3:category.find("(")]
        
        # all images in that label
        for curr_img_nm in os.listdir(label_img_path):
            # reading image
            curr=cv2.imread(os.path.join(label_img_path,curr_img_nm))
            new=seq(images=curr)
                            
            # saving new image file in folder
            name=str(curr_img_nm).replace('.jpg',sub_nm+'.jpg')
            cv2.imwrite(os.path.join(aug_img_path,label,name),new)


category :  Fliplr(name=UnnamedFliplr, parameters=[Deterministic(int 1)], deterministic=False)


S:\object detection\TFODCourse\tfod\lib\site-packages\imgaug\augmenters\base.py:27: SuspiciousMultiImageShapeWarning: You provided a numpy array of shape (480, 640, 3) as a multi-image augmentation input, which was interpreted as (N, H, W). The last dimension however has value 1 or 3, which indicates that you provided a single image with shape (H, W, C) instead. If that is the case, you should use e.g. augmenter(image=<your input>) or augment_image(<your input>) -- note the singular 'image' instead of 'imageS'. Otherwise your single input image will be interpreted as multiple images of shape (H, W) during augmentation.
  ia.warn(


category :  GaussianBlur(name=UnnamedGaussianBlur, parameters=[Uniform(Deterministic(int 0), Deterministic(int 1))], deterministic=False)
category :  LinearContrast(name=UnnamedLinearContrast, parameters=[Uniform(Deterministic(float 0.75000000), Deterministic(float 1.50000000))], deterministic=False)
category :  Multiply(name=UnnamedMultiply, parameters=[Uniform(Deterministic(float 0.80000000), Deterministic(float 1.20000000)), Deterministic(int 1)], deterministic=False)
category :  Flipud(name=UnnamedFlipud, parameters=[Deterministic(int 1)], deterministic=False)
category :  Sharpen(name=UnnamedSharpen, parameters=[<imgaug.augmenters.convolutional._SharpeningMatrixGenerator object at 0x000001F586771420>, function], deterministic=False)
category :  Invert(name=UnnamedInvert, parameters=[Deterministic(int 1), Deterministic(int 1), None, None, None, Binomial(Deterministic(float 0.50000000))], deterministic=False)
category :  Crop(name=UnnamedCrop, parameters=[DiscreteUniform(Deterministi


# Labeling data

In [7]:
!pip install --upgrade pyqt5 lxml

In [5]:
Label_img_path = os.path.join(root_path,'labelimg')

if not os.path.exists(Label_img_path):
    !mkdir {Label_img_path}
    !git clone https://github.com/tzutalin/labelImg {Label_img_path}

In [6]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {Label_img_path} && pyrcc5 -o libs/resources.py resources.qrc
    
!cd {Label_img_path} && python labelImg.py

# Dividing in training and testing data



In [20]:
# creating train and test folder

if not os.path.exists(train_path):
    !mkdir {train_path}
if not os.path.exists(test_path):
    !mkdir {test_path}

In [6]:
import shutil
import random

In [ ]:
# moving the 60% images from augmented folder to training folder and 40% to test folder
for label in labels:
    # path of image folder
    label_img_path=os.path.join(aug_img_path,label)

    # getting name of each file in that directory 
    lst=list(os.listdir(label_img_path))
    random.shuffle(lst)
#     print(lst[10:30])
    for i in range(0,999):
        try: 
            shutil.move(os.path.join(label_img_path,lst[i]),os.path.join(train_path,lst[i]))
        except: 
            print("error")
            pass
    for i in range(1000,1520):
        try: 
            shutil.move(os.path.join(label_img_path,lst[i]),os.path.join(test_path,lst[i]))
        except: 
            print("error")
            pass

In [8]:
# dividing test into test and validation 
# getting name of each file in that directory 
lst=list(os.listdir(test_path))
random.shuffle(lst)
for i in range(0,6791):
    try: 
        shutil.move(os.path.join(test_path,lst[i]),os.path.join(validate_path,lst[i]))
    except: 
        print("error")
        pass

In [32]:
# archive_path=r"S:\object detection\archive\data"
for label in labels:
    # path of image folder
    path=os.path.join(archive_path,label)

    # getting name of each file in that directory 
    lst=list(os.listdir(path))
    print("train",label)
    for i in range(0,899):
        data = os.path.splitext(lst[i])
        only_name = data[0]
        extension = data[1]
        # Adding the new name
        new_name = only_name + "_label_"+str(label) + extension 
        shutil.copy(os.path.join(path,lst[i]),os.path.join(train_path,new_name))
    for i in range(900,1199):
        data = os.path.splitext(lst[i])
        only_name = data[0]
        extension = data[1]
        # Adding the new name
        new_name = only_name + "_label_"+str(label) + extension
        shutil.copy(os.path.join(path,lst[i]),os.path.join(test_path,new_name))

train 1
train 2
train 3
train 4
train 5
train 6
train 7
train 8
train 9
train A
train B
train C
train D
train E
train F
train G
train H
train I
train J
train K
train L
train M
train N
train O
train P
train Q
train R
train S
train T
train U
train V
train W
train X
train Y
train Z


In [10]:
f_labels="S:\object detection\TFODCourse\ISL\workspace\labels"
f_done="S:\object detection\TFODCourse\ISL\workspace\done"
n_labels=os.listdir(f_labels)
n_done=os.listdir(f_done)
print(len(n_labels),len(n_done))
print(n_labels[4],n_done[4])

24940 25497
08.json 08.jpg


In [47]:
l1=[]
l2=[]
for i in n_labels:
    data = os.path.splitext(i)
    nm=data[0]
    l1.append(nm)
print(len(l1))

24940


In [48]:
for i in n_done:
    data = os.path.splitext(i)
    nm=data[0]
    l2.append(nm)
print(len(l2))

25497


In [67]:
set_labels = set(l1)
set_done = set(l2)
diff = set_labels - set_done
print(len(diff),diff)

113 {'1195', '871', '841', '916', '846', '861', '854', '1175', '863', '838', '1190', '1152', '1164', '834', '835', '857', '1166', '920', '873', '919', '874', '1155', '917', '1186', '1167', '837', '1192', '862', '921', '1160', '864', '903', '905', '872', '858', '915', '1199', '830', '901', '1159', '1177', '1174', '855', '829', '1191', '1189', '848', '839', '1153', '1193', '870', '1165', '1173', '850', '875', '1154', '1196', '1178', '833', '840', '1198', '865', '860', '856', '842', '1187', '1162', '1176', '1197', '1194', '912', '832', '831', '859', '853', '843', '844', '922', '1172', '913', '867', '900', '1157', '836', '1163', '851', '914', '868', '1184', '906', '1171', '1161', '923', '910', '907', '908', '1156', '852', '847', '1188', '904', '911', '1169', '866', '1168', '909', '902', '1170', '849', '918', '869', '845', '1158'}


In [ ]:
import shutil
x=r"S:\object detection\TFODCourse\ISL\workspace\train robo"
yd=r"S:\object detection\TFODCourse\ISL\workspace\labels"
diff=list(diff)
print(type(diff))
for i in range(0,len(list(diff))):
    nm=diff[i]+".json"
    shutil.move(os.path.join(yd,nm),os.path.join(x,nm))
    print("done",diff[i])